In [8]:
#Importation des bibliothèques

from dotenv import load_dotenv
import os
import sys
import requests
import json
import logging
import time
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
from io import StringIO
import seaborn as sns
import folium
import json
import requests
import time

In [9]:
import fonctions as f

df = f.pd.read_csv("data.csv", index_col=0)
df

,DATE,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,TNSOL,...,BRUME,ECLAIR,ETPMON,ETPGRILLE,UV,TMERMAX,TMERMIN,HNEIGEF,NEIGETOTX,NEIGETOT06
POSTE,,,,,,,,,,,,,,,,,,,,,
1014002,20220105,"2,6",NaN,"-0,4",1759.0,"3,4",1217.0,"0,9","1,50",NaN,...,NaN,NaN,NaN,"0,4",NaN,NaN,NaN,NaN,0.0,0.0
1014002,20220106,"0,0",NaN,"-2,5",1746.0,"4,3",1327.0,"-0,6","0,90",NaN,...,NaN,NaN,NaN,"0,6",NaN,NaN,NaN,NaN,0.0,0.0
1014002,20220107,"10,0",NaN,"-6,9",718.0,"1,6",2229.0,"-1,2","-2,65",NaN,...,NaN,NaN,NaN,"0,0",NaN,NaN,NaN,NaN,0.0,NaN
1014002,20220108,"13,2",NaN,"-0,7",733.0,"6,3",2319.0,"2,4","2,80",NaN,...,NaN,NaN,NaN,"0,5",NaN,NaN,NaN,NaN,0.0,0.0
1014002,20220109,"7,6",NaN,"0,4",737.0,"2,2",601.0,"1,8","1,30",NaN,...,NaN,NaN,NaN,"0,1",NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014002,20241027,"0,2",NaN,"5,7",633.0,"21,9",1354.0,"13,3","13,80",NaN,...,NaN,NaN,NaN,"1,1",NaN,NaN,NaN,NaN,0.0,0.0
1014002,20241028,"0,2",NaN,"6,7",546.0,"21,9",1236.0,"12,6","14,30",NaN,...,NaN,NaN,NaN,"1,3",NaN,NaN,NaN,NaN,0.0,0.0
1014002,20241029,"0,4",NaN,"6,4",2251.0,"21,7",1236.0,"12,0","14,05",NaN,...,NaN,NaN,NaN,"1,1",NaN,NaN,NaN,NaN,0.0,0.0
